# Evaluación de la relevancia funcional de las parejas SL mediante STRING
```
Autor:  Martín Sende Pombo (email: martinsendepombo@outlook.com)
Este cuaderno de Jupyter se basa en la documentación para desarrolladores sobre la API (Application Programming Interface) de STRING (Search Tool for the Retrieval of Interacting Genes/Proteins), dispoble en: https://string-db.org/help/api/#string-api
Se utilizó ChatGPT 3.5 como asistente de programación. 
Creado: 09-05-2024
Última modificación: 02-07-2024
Propósito: Este cuaderno de Jupyter permite emplear las herramientas de la API de STRING, útiles para determinar si existen interacciones entre las proteínas codificadas por el gen de entrada de una firma de expresión génica, y cada uno de sus pares SL predichos computacionalmente.
```
Copyright (C) 2024  Martín Sende Pombo

        Este programa es software libre: puede redistribuirlo y/o modificarlo
        bajo los términos de la Licencia Pública General de GNU publicada por la
        Free Software Foundation, ya sea la versión 3 de la Licencia,
        o (a su elección) cualquier versión posterior.

        Este programa se distribuye con la esperanza de que sea útil,
        pero SIN NINGUNA GARANTÍA; ni siquiera la garantía implícita
        de COMERCIABILIDAD o IDONEIDAD PARA UN PROPÓSITO PARTICULAR.
        Consulte la Licencia Pública General GNU para más detalles.

        Debería haber recibido una copia de la Licencia Pública General GNU
        junto con este programa. Si no es así, consulte <https://www.gnu.org/licenses/>.

In [1]:
!pip3 install requests

In [2]:
!pip3 install openpyxl

## Versión de STRING
Versión estable más reciente: https://string-db.org/api/json/version

In [3]:
#Versión de STRING seleccionada
string_api_url = "https://version-12-0.string-db.org/api"

## Identificador NCBI de la especie
Uno de los parámetros que resulta más útil definir para interaccionar con STRING, a través de su API, es el correspondiente a los identificadores de los taxones asignados por el NCBI a las especies (véase el parámetro 'species'). Por ejemplo, el del ser humano (Homo sapiens) es 9606. Puede consultarse el de otros organismos presentes en STRING [aquí](https://string-db.org/cgi/input.pl?input_page_active_form=organisms). Alternativamente, puede buscarse este ID en el [Taxonomy Browser](https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi) del NCBI.

Al especificar de qué especie proceden sus proteínas se garantiza una respuesta más rápida del servidor. Además, la API rechazará consultas para redes de más de 10 proteínas sin el organismo especificado.

## Asignación de identificadores
Puede llamar a nuestra API STRING con nombres comunes de genes, varios sinónimos o incluso identificadores UniProt y números de acceso. Sin embargo, STRING no siempre los entiende, lo que puede dar lugar a errores o incoherencias. Antes de utilizar otros métodos de la API siempre es ventajoso asignar sus identificadores a los que utiliza STRING. Además, STRING resolverá sus propios identificadores más rápidamente, por lo que su herramienta/sitio web verá un beneficio en velocidad si los utiliza. Para cada proteína de entrada STRING coloca el identificador que mejor coincide en la primera fila, por lo que la primera línea será normalmente la correcta.

Para más información pulse [aquí](https://string-db.org/cgi/help.pl?subpage=api%23mapping-identifiers).

### Formatos de salida disponibles:

| Formato | Descripción |
|:--------|:------------|
| `tsv`| Valores separados por tabuladores, con una línea de encabezamiento |
| `tsv-no-header` | Valores separados por tabuladores, sin línea de encabezamiento |
| `json` | Formato JSON |
| `xml` | Formato XML |

### Parámetros disponibles:

| Parámetro | Descripción |
|:------|:------------|
| `identifiers`| Parámetro obligatorio para varios elementos, por ejemplo, DRD1_HUMAN%0dDRD2_HUMAN |
| `echo_query` | Inserta una columna con su identificador de entrada (toma los valores '0' o '1', por defecto es '0') |
| `limit` | Limita el número de coincidencias por identificador de consulta (las mejores coincidencias van primero) |
| `species` | Identificadores de taxones del NCBI (por ejemplo, Humano es 9606, véase: [organismos de STRING](https://string-db.org/cgi/input.pl?input_page_active_form=organisms)) |
| `caller_identity`| Su identificador para la API de STRING |

### Ejemplo de código python3

In [12]:
#!/usr/bin/env python3

##########################################################
## Para una lista dada de proteínas el script las resuelve
## (si es posible) al identificador STRING que mejor coincida
## e imprime el mapeo en pantalla en formato TSV
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la línea de comandos
## (win) o terminal (mac/linux) para instalar el módulo
###########################################################

import requests ## python -m pip install requests

string_api_url = "https://version-11-5.string-db.org/api"
output_format = "tsv-no-header"
method = "get_string_ids"

##
## Establecer parámetros
##

params = {

    "identifiers" : "\r".join(["p53", "BRCA1", "cdk2", "Q99835"]), # tu lista de proteínas
    "species" : 9606, # identificador NCBI de la especie 
    "limit" : 1, # sólo un (el mejor) identificador por proteína de entrada
    "echo_query" : 1, # ver sus identificadores de entrada en la salida
    "caller_identity" : "www.awesome_app.org" # nombre de su aplicación

}

##
## Construir URL
##


request_url = "/".join([string_api_url, output_format, method])

##
## Llamar a STRING
##

results = requests.post(request_url, data=params)

##
## Leer y analizar los resultados
##

for line in results.text.strip().split("\n"):
    l = line.split("\t")
    input_identifier, string_identifier = l[0], l[2]
    print("Input:", input_identifier, "STRING:", string_identifier, sep="\t")

Input:	p53	STRING:	9606.ENSP00000269305
Input:	BRCA1	STRING:	9606.ENSP00000418960
Input:	cdk2	STRING:	9606.ENSP00000266970
Input:	Q99835	STRING:	9606.ENSP00000249373


### Código a ejecutar

In [7]:
#!/usr/bin/env python3

##########################################################
## Para una lista dada de proteínas el script las resuelve
## (si es posible) al identificador STRING que mejor coincida
## e imprime el mapeo en pantalla en formato TSV
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la línea de comandos
## (win) o terminal (mac/linux) para instalar el módulo
###########################################################
import os
import csv
#!pip3 install requests
import requests ## python -m pip install requests
from time import sleep

#string_api_url = "https://version-12-0.string-db.org/api"
output_format = "tsv-no-header"
method = "get_string_ids"

def read_identifiers_from_rnk(file_path):
    identifiers = []
    with open(file_path, 'r') as file:
        for line in file:
            identifier = line.strip().split('\t')[0]
            identifiers.append(identifier)
    return identifiers

def read_identifiers_from_csv(file_path):
    identifiers = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            identifier = row[0]
            identifiers.append(identifier)
    return identifiers

def process_files_in_folder(folder_path, file_extension):
    file_identifiers_map = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith(file_extension):
            file_path = os.path.join(folder_path, file_name)
            if file_extension == '.rnk':
                identifiers = read_identifiers_from_rnk(file_path)
            elif file_extension == '.csv':
                identifiers = read_identifiers_from_csv(file_path)
            file_identifiers_map[file_name] = identifiers
    return file_identifiers_map

# Solicitar la carpeta que contiene los archivos
folder_path = 'Firmas_genes'

# Solicitar el tipo de archivo
file_extension = input("¿Qué tipo de archivo son los identificadores: .rnk o .csv? ")

if file_extension not in ['.rnk', '.csv']:
    print("Tipo de archivo no válido. Debe ser .rnk o .csv.")
    exit()

# Procesar los archivos en la carpeta especificada
file_identifiers_map = process_files_in_folder(folder_path, file_extension)

##
## Establecer parámetros
##

params = {
    "species": 9606,  # identificador NCBI de la especie (p.ej., el del humano es 9606)
    "limit": 1,  # sólo un (el mejor) identificador por proteína de entrada
    "echo_query": 1,  # ver sus identificadores de entrada en la salida
    "caller_identity": "www.facebook.com/MSPCiencias/"  # nombre de su aplicación
}

##
## Construir URL
##

request_url = "/".join([string_api_url, output_format, method])

##
## Llamar a STRING y guardar los resultados en archivos .txt
##

output_folder = 'Identificadores_STRING'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for file_name, identifiers in file_identifiers_map.items():
    params["identifiers"] = "\r\n".join(identifiers)
    results = requests.post(request_url, data=params)
    output_file = os.path.join(output_folder, file_name.replace(file_extension, '_identificadores_STRING.txt'))
    with open(output_file, 'w') as f:
        for line in results.text.strip().split("\n"):
            l = line.split("\t")
            string_identifier = l[2]
            f.write(f"{string_identifier}\n")
    # Esperar un segundo antes de la siguiente petición
    sleep(1)

print("Los identificadores de STRING han sido guardados en archivos en la carpeta:", output_folder)

¿Qué tipo de archivo son los identificadores: .rnk o .csv?  .csv


Los identificadores de STRING han sido guardados en archivos en la carpeta: Identificadores_STRING


### Campos de salida:

| Campo | Descripción |
|:------|:------------|
| `queryItem`| (OPCIONAL) Su proteína de entrada |
| `queryIndex` | Posición de la proteína en su entrada (empezando por la posición 0) |
| `stringId` | Identificador de STRING |
| `ncbiTaxonId` | Identificador de taxón del NCBI |
| `taxonName`| Nombre de la especie |
| `preferredName`| Nombre común de la proteína |
| `annotation`| Anotación de la proteína |

## Obtener el enriquecimiento de la interacción proteína-proteína (opcional)
Incluso en ausencia de proteínas anotadas (por ejemplo, en genomas nuevos) STRING puede decirle si su subconjunto de proteínas está relacionado funcionalmente, es decir, si está enriquecido en interacciones en comparación con la distribución de interacciones de fondo en todo el proteoma. La descripción detallada del método de enriquecimiento PPI puede encontrarse [aquí](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3531103/).

Para información adicional pulse [aquí](https://string-db.org/cgi/help.pl?subpage=api%23getting-protein-protein-interaction-enrichment).

### Formatos de salida disponibles:

| Formato | Descripción |
|:--------|:------------|
| `tsv`| Valores separados por tabuladores, con una línea de encabezamiento |
| `tsv-no-header` | Valores separados por tabuladores, sin línea de encabezamiento |
| `json` | Formato JSON |
| `xml` | Formato XML |

### Parámetros disponibles:

| Parámetro | Descripción |
|:------|:------------|
| `identifiers`| Parámetro obligatorio para varios elementos, por ejemplo, DRD1_HUMAN%0dDRD2_HUMAN |
| `species` | Identificadores de taxones del NCBI (por ejemplo, Humano es 9606, véase: [organismos de STRING](https://string-db.org/cgi/input.pl?input_page_active_form=organisms)) |
| `required_score` | Umbral de significación para incluir una interacción, un número entre 0 y 1000 (por defecto depende de la red) |
| `background_string_identifiers` | Con este parámetro puede especificar el proteoma de fondo de su experimento. Sólo se reconocerán los identificadores STRING (cada uno debe ir separado por "%0d"), por ejemplo '7227.FBpp0077451%0d7227.FBpp0074373'. Puede asignar identificadores STRING utilizando el método de [asignación de identificadores](https://string-db.org/cgi/help.pl?subpage=api%23mapping-identifiers). |
| `caller_identity`| Su identificador para la API de STRING |

### Ejemplo de código python3

In [ ]:
#!/usr/bin/env python

##############################################################
## El script imprime el valor p de STRING proteína-proteína
## método de enriquecimiento de interacciones para el conjunto dado de proteínas 
##
## Requiere el módulo de solicitudes:
## escriba "python -m pip install requests" en la línea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
##############################################################

import requests ## python -m pip install requests

string_api_url = "https://version-11-5.string-db.org/api"
output_format = "tsv-no-header"
method = "ppi_enrichment"

##
## Construir la solicitud
##

request_url = "/".join([string_api_url, output_format, method])

##
## Establecer parámetros
##

my_genes = ['7227.FBpp0074373', '7227.FBpp0077451', '7227.FBpp0077788',
            '7227.FBpp0078993', '7227.FBpp0079060', '7227.FBpp0079448']

params = {

    "identifiers" : "%0d".join(my_genes), # su/s proteína/s
    "species" : 7227, # identificador NCBI de la especie  
    "caller_identity" : "www.awesome_app.org" # nombre de su aplicación

}

##
## Llamar a STRING
##

response = requests.post(request_url, data=params)

##
## Analiza e imprime la respuesta
##

for line in response.text.strip().split("\n"):
    pvalue = line.split("\t")[5]
    print("P-value:", pvalue)

### Código a ejecutar

In [90]:
#!/usr/bin/env python

##############################################################
## El script imprime el valor p de STRING proteína-proteína
## método de enriquecimiento de interacciones para el conjunto dado de proteínas 
##
## Requiere el módulo de solicitudes:
## escriba "python -m pip install requests" en la línea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
##############################################################
import os
#!pip3 install requests
import requests ## python -m pip install requests
import time

# Directorio de entrada y salida
input_directory = "Identificadores_STRING"
output_directory = "Resultados_EIPP"

# Crear el directorio de salida si no existe
os.makedirs(output_directory, exist_ok=True)

# Obtener la lista de archivos de entrada
input_files = os.listdir(input_directory)

# URL de la API de STRING
#string_api_url = "https://version-12-0.string-db.org/api"

# Formato de salida y método
output_format = "tsv-no-header"
method = "ppi_enrichment"

# Iterar sobre cada archivo de entrada
for input_file in input_files:
    # Construir la ruta completa del archivo de entrada
    input_file_path = os.path.join(input_directory, input_file)
    
    # Leer los identificadores del archivo
    with open(input_file_path, "r") as file:
        my_genes = file.read().strip().split("\t")
    
    # Construir la solicitud
    request_url = "/".join([string_api_url, output_format, method])
    
    # Establecer parámetros
    params = {
        "identifiers" : "%0d".join(my_genes), # su/s proteína/s
        "species" : 9606, # identificador NCBI de la especie (p.ej., el del humano es 9606)  
        "caller_identity": "www.facebook.com/MSPCiencias/" # nombre de su aplicación
    }
    
    # Llamar a STRING
    response = requests.post(request_url, data=params)
    
    # Construir el nombre del archivo de salida
    base_name = os.path.splitext(input_file)[0].replace("_identificadores_STRING", "")
    output_file_name = base_name + "_EIPP.txt"
    output_file_path = os.path.join(output_directory, output_file_name)
    
    # Procesar la respuesta y escribir en el archivo de salida
    with open(output_file_path, "w") as output_file:
        for line in response.text.strip().split("\n"):
            columns = line.split("\t")
            if len(columns) >= 6:
                number_of_nodes = columns[0]
                number_of_edges = columns[1]
                average_node_degree = columns[2]
                local_clustering_coefficient = columns[3]
                expected_number_of_edges = columns[4]
                pvalue = columns[5]
    
                output_file.write("Número de nodos:\t{}\n".format(number_of_nodes))
                output_file.write("Número de aristas:\t{}\n".format(number_of_edges))
                output_file.write("Grado medio de los nodos:\t{}\n".format(average_node_degree))
                output_file.write("Coeficiente de agrupamiento local:\t{}\n".format(local_clustering_coefficient))
                output_file.write("Número de aristas esperado:\t{}\n".format(expected_number_of_edges))
                output_file.write("Valor p:\t{}\n".format(pvalue))
            else:
                output_file.write("Error: La línea no tiene suficientes columnas.\n")
    
    # Pausa de 1 segundo antes de procesar el siguiente archivo
    time.sleep(1)
    
print("Proceso completado.")

Proceso completado.


### Campos de salida:

| Campo | Descripción |
|:------|:------------|
| `number_of_nodes`| Número de proteínas en tu red |
| `number_of_edges` | Número de aristas en la red |
| `average_node_degree` | Grado medio de los nodos de la red |
| `local_clustering_coefficient` | Coeficiente medio de agrupamiento local |
| `expected_number_of_edges`| Número esperado de aristas en función de los grados de los nodos |
| `p_value`| Significación de que su red tenga más interacciones de las esperadas |

## Obtención de las interacciones de la red STRING
El método API de red también le permite recuperar su red de interacción STRING para una o múltiples proteínas en varios formatos de texto. Le indicará la puntuación combinada y todas las puntuaciones específicas del canal para el conjunto de proteínas. También puede ampliar el vecindario de la red configurando "add_nodes", que añadirá, a su red, nuevos socios de interacción en orden de confianza.

Para más información pulse [aquí](https://string-db.org/cgi/help.pl?subpage=api%23getting-the-string-network-interactions).

### Formatos de salida disponibles:

| Formato | Descripción |
|:--------|:------------|
| `tsv`| Valores separados por tabuladores, con una línea de encabezamiento |
| `tsv-no-header` | Valores separados por tabuladores, sin línea de encabezamiento |
| `json` | Formato JSON |
| `xml` | Formato XML |
| `psi-mi` | Formato XML PSI-MI |
| `psi-mi-tab` | Formato PSI-MITAB |

### Parámetros disponibles:

| Parámetro | Descripción |
|:------|:------------|
| `identifiers`| Parámetro obligatorio para varios elementos, por ejemplo, DRD1_HUMAN%0dDRD2_HUMAN |
| `species` | Identificadores de taxones del NCBI (por ejemplo, Humano es 9606, véase: [organismos de STRING](https://string-db.org/cgi/input.pl?input_page_active_form=organisms)) |
| `required_score` | Umbral de significación para incluir una interacción, un número entre 0 y 1000 (por defecto depende de la red) |
| `network_type` | Tipo de red: funcional (por defecto), física |
| `add_nodes`| Añade un número de proteínas a la red en función de su puntuación de confianza |
| `show_query_node_labels`| Cuando está habilitado, utiliza los nombres enviados en la columna preferredName cuando (0 o 1) (por defecto:0) |
| `caller_identity`| Su identificador para la API de STRING |

Si consulta la API con una proteína, el parámetro "add_nodes" se establece automáticamente en 10, de modo que puede obtener el vecindario de interacciones de su proteína de consulta. Sin embargo, al igual que en la página web de STRING, si consulta la API con más de una proteína, el método sólo mostrará las interacciones entre las proteínas de entrada. Por supuesto, siempre puede ampliar el vecindario de interacciones ajustando el parámetro "add_nodes" al valor deseado.

### ¿Cómo puedo seleccionar un valor de corte razonable para mi análisis?

Puede utilizar la puntuación de corte para limitar el número de interacciones a las que tienen mayor confianza y más probabilidades de ser verdaderos positivos. Establecer el límite más bajo aumentará la cobertura, pero también la fracción de falsos positivos. Debe elegir un número arbitrario basado en el número de interacciones que necesita para su análisis.

Fuente de información: https://string-db.org/help//faq/#how-do-i-select-a-reasonable-score-cut-off-value-for-my-analysis

### Ejemplo de código python3

In [7]:
#!/usr/bin/env python3

##################################################################
## Para la lista dada de proteinas imprime solo las interacciones
## entre estas proteínas que tienen una confianza media o alta
## puntuación experimental
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la linea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
##################################################################

import requests ## python -m pip install requests


string_api_url = "https://version-11-5.string-db.org/api"
output_format = "tsv-no-header"
method = "network"

##
## Construir URL
##

request_url = "/".join([string_api_url, output_format, method])

##
## Establecer parámetros
##

my_genes = ["CDC42","CDK1","KIF23","PLK1",
            "RAC2","RACGAP1","RHOA","RHOB"]

params = {

    "identifiers" : "%0d".join(my_genes), # su/s proteína/s
    "species" : 9606, # identificador NCBI de la especie 
    "caller_identity" : "www.awesome_app.org" # nombre de su aplicación

}

##
## Llamar a STRING
##

response = requests.post(request_url, data=params)

for line in response.text.strip().split("\n"):

    l = line.strip().split("\t")
    p1, p2 = l[2], l[3]

    ## filtrar la interacción según la puntuación experimental
    experimental_score = float(l[10])
    if experimental_score > 0.4:
        ## print 
        print("\t".join([p1, p2, "experimentally confirmed (prob. %.3f)" % experimental_score]))

RAC2	CDC42	experimentally confirmed (prob. 0.481)
RAC2	CDC42	experimentally confirmed (prob. 0.481)
KIF23	RACGAP1	experimentally confirmed (prob. 0.993)
KIF23	RACGAP1	experimentally confirmed (prob. 0.993)
KIF23	PLK1	experimentally confirmed (prob. 0.621)
KIF23	PLK1	experimentally confirmed (prob. 0.621)
PLK1	RACGAP1	experimentally confirmed (prob. 0.721)
PLK1	RACGAP1	experimentally confirmed (prob. 0.721)
CDC42	RHOA	experimentally confirmed (prob. 0.705)
CDC42	RHOA	experimentally confirmed (prob. 0.705)
CDC42	RACGAP1	experimentally confirmed (prob. 0.820)
CDC42	RACGAP1	experimentally confirmed (prob. 0.820)


### Código a ejecutar

In [91]:
#!/usr/bin/env python3

##################################################################
## Para la lista dada de proteinas imprime solo las interacciones
## entre estas proteínas que tienen una confianza media o alta
## puntuación experimental
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la linea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
##################################################################
import os
#!pip3 install requests
import requests
#!pip3 install openpyxl
import openpyxl
import time

# Carpeta de entrada y salida
input_folder = "Identificadores_STRING"
output_folder = "Interacciones_red"

# Verificar si la carpeta de salida existe, si no, crearla
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Obtener la lista de archivos .txt en la carpeta de entrada
txt_files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]

# Iterar sobre cada archivo .txt
for txt_file in txt_files:
    try:
        # Leer los identificadores del archivo .txt
        file_path = os.path.join(input_folder, txt_file)
        with open(file_path, "r") as file:
            my_genes = file.read().strip().split("\t")

        # Definir la URL de la API de STRING
        #string_api_url = "https://version-12-0.string-db.org/api"
        output_format = "tsv-no-header"
        method = "network"
        request_url = "/".join([string_api_url, output_format, method])

        # Establecer parámetros
        params = {
            "identifiers": "%0d".join(my_genes), # su/s proteína/s
            "species": 9606, # identificador NCBI de la especie (p.ej., el del humano es 9606) 
            "caller_identity": "www.facebook.com/MSPCiencias/" # nombre de su aplicación
        }

        # Llamar a STRING
        response = requests.post(request_url, data=params)

        # Verificar si la respuesta fue exitosa
        response.raise_for_status()

        # Crear un nuevo archivo Excel
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["ID STRING PA", "Proteína A", "Proteína B", "Confirmación experimental", "Probabilidad", "Interacciones de alta confianza", "Puntuación combinada"])

        # Procesar la respuesta y escribir en el archivo Excel
        for line in response.text.strip().split("\n"):
            l = line.strip().split("\t")
            if len(l) >= 13:  # Verificar si hay suficientes columnas en la línea
                stringId_A, preferredName_A, preferredName_B = l[0], l[2], l[3]
                escore = float(l[10])
                confirmation = "Sí" if escore > 0.5 else "No"
                score = float(l[5])
                high_confidence = "Sí" if score > 0.7 else "No"
                ws.append([stringId_A, preferredName_A, preferredName_B, confirmation, "%.3f" % escore, high_confidence, "%.3f" % score])

        # Guardar el archivo Excel
        excel_file_name = txt_file.replace("_identificadores_STRING.txt", "_interacciones_red.xlsx")
        excel_file_path = os.path.join(output_folder, excel_file_name)
        wb.save(excel_file_path)
        print("Los resultados del archivo {} se han guardado en el archivo Excel: {}".format(txt_file, excel_file_path))
    except Exception as e:
        print("Se produjo un error al procesar el archivo {}: {}".format(txt_file, str(e)))
        continue

    # Hacer una pausa de 1 segundo antes de procesar el siguiente archivo
    time.sleep(1)

print("Proceso completado.")

Los resultados del archivo ACOT9_2_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\ACOT9_2_interacciones_red.xlsx
Los resultados del archivo MMP1_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\MMP1_interacciones_red.xlsx
Los resultados del archivo MMP3_2_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\MMP3_2_interacciones_red.xlsx
Los resultados del archivo NETO2_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\NETO2_interacciones_red.xlsx
Los resultados del archivo PSMB2_2_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\PSMB2_2_interacciones_red.xlsx
Los resultados del archivo SLC16A1_2_identificadores_STRING.txt se han guardado en el archivo Excel: Interacciones_red\SLC16A1_2_interacciones_red.xlsx
Proceso completado.


### Campos de salida (en formatos TSV y JSON):

| Campo | Descripción |
|:------|:------------|
| `stringId_A` | Identificador de STRING (proteína A) |
| `stringId_B` | Identificador de STRING (proteína B) |
| `preferredName_A` | Nombre común de la proteína (proteína A) |
| `preferredName_B` | Nombre común de la proteína (proteína B) |
| `ncbiTaxonId` | Identificador de taxón del NCBI |
| `score` | Puntuación combinada |
| `nscore` | Puntuación de vecindad de genes |
| `fscore` | Puntuación de fusión de genes |
| `pscore` | Puntuación del perfil filogenético |
| `ascore` | Puntuación de coexpresión |
| `escore` | Puntuación experimental |
| `dscore` | Puntuación de la base de datos |
| `tscore` | Puntuación de minería de textos |

### ¿Qué significan las columnas de puntuación (por ejemplo, nscore, fscore, tscore, etc.)?

He aquí un resumen.
* nscore - puntuación de vecindad, (calculada a partir del recuento de nucleótidos entre genes).
* fscore - puntuación de fusión (derivada de proteínas fusionadas en otras especies).
* pscore - puntuación de coocurrencia del perfil filético (derivada de patrones similares de ausencia/presencia de genes).
* hscore - puntuación de homología, el grado de homología de los interactores (normalmente no se informa en STRING).
* ascore - puntuación de coexpresión (derivada de patrones similares de expresión de ARNm medidos mediante matrices de ADN y tecnologías similares).
* escore - puntuación experimental (derivada de datos experimentales, como cromatografía de afinidad).
* dscore - puntuación de base de datos (derivada de datos curados de varias bases de datos).
* tscore - puntuación de minería de textos (derivada de la co-ocurrencia de nombres de genes/proteínas en resúmenes).

### ¿Cómo se calculan las puntuaciones?

La puntuación combinada (score) se calcula combinando las probabilidades de los distintos canales de evidencia y se corrige para tener en cuenta la probabilidad de observar aleatoriamente una interacción. Para una descripción más detallada, véase [von Mering, et al. Nucleic Acids Res. 2005](https://www.ncbi.nlm.nih.gov/pubmed/15608232).

Para combinar las puntuaciones se suman las probabilidades de cada uno de los canales. A cada canal se le ha añadido un 'prior' para tener en cuenta la probabilidad de que dos proteínas elegidas al azar estén interactuando. Antes de combinar los canales, hay que eliminar el "prior" y volver a añadirlo a la puntuación combinada. Así es como se calcula la puntuación combinada para una interacción. De todos modos, dispone de más información en la sección de preguntas frecuentes [(FAQ)](https://string-db.org/help//faq/#how-are-the-scores-computed).

### ¿Cómo extraer interacciones de alta confianza (>0.7)?
Tenga en cuenta que las puntuaciones se multiplican por 1000 para hacerlos enteros.

Fuente: https://string-db.org/help//faq/#how-to-extract-high-confidence-07-interactions-from-information-on-combined-score-in-proteinlinkstxtgz

In [118]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos .xlsx
folder_path = 'Interacciones_red'

# Obtener la lista de todos los archivos .xlsx en la carpeta
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

for file in files:
    # Leer el archivo .xlsx
    input_path = os.path.join(folder_path, file)
    df = pd.read_excel(input_path)

    # Filtrar las filas que cumplen las condiciones especificadas
    filtered_df = df[(df['Probabilidad'] > 0.5) & (df['Puntuación combinada'] >= 0.7)]

    # Eliminar las columnas "Confirmación experimental" e "Interacciones de alta confianza"
    filtered_df = filtered_df.drop(columns=['Confirmación experimental', 'Interacciones de alta confianza'])

    # Renombrar la columna "Probabilidad" a "Probabilidad confirmación experimental"
    filtered_df = filtered_df.rename(columns={'Probabilidad': 'Probabilidad confirmación experimental'})

    # Eliminar las filas que ocupan una posición par (índice par)
    filtered_df = filtered_df.iloc[1::2]

    # Verificar si el DataFrame resultante no está vacío antes de guardarlo
    if not filtered_df.empty:
        # Crear el nombre del archivo de salida
        output_file_name = file.replace('.xlsx', '_AC.xlsx')
        output_path = 'Interacciones_alta_confianza'
        output_path = os.path.join(output_path, output_file_name)

        # Escribir el DataFrame resultante a un nuevo archivo .xlsx
        filtered_df.to_excel(output_path, index=False)

        print(f"El archivo contenía interacciones de alta confianza y ha sido guardado en: {output_path}")
    else:
        print(f"No se generó un nuevo archivo para {file} porque no contenía interacciones de alta confianza.")
        
print("Proceso completado.")

No se generó un nuevo archivo para SLC5A8_2_interacciones_red.xlsx porque no contenía interacciones de alta confianza.
Proceso completado.


### Identificación de posibles parejas SL con relevancia funcional

In [81]:
import os
import pandas as pd

# Directorios de entrada y salida
input_directory = 'Interacciones_alta_confianza'
output_directory = 'Interacciones_interes'

# Crear el directorio de salida si no existe
os.makedirs(output_directory, exist_ok=True)

# Procesar cada archivo en la carpeta de entrada
for filename in os.listdir(input_directory):
    if filename.endswith('_interacciones_red_AC.xlsx') or filename.endswith('_2_interacciones_red_AC.xlsx'):
        # Obtener el nombre base del archivo sin la terminación
        if filename.endswith('_2_interacciones_red_AC.xlsx'):
            base_name = filename.replace('_2_interacciones_red_AC.xlsx', '')
        else:
            base_name = filename.replace('_interacciones_red_AC.xlsx', '')

        # Leer el archivo Excel
        file_path = os.path.join(input_directory, filename)
        df = pd.read_excel(file_path)

        # Filtrar las filas que contengan la cadena en las columnas "Proteína A" o "Proteína B"
        filtered_df = df[(df['Proteína A'].str.contains(base_name, na=False)) | (df['Proteína B'].str.contains(base_name, na=False))]

        # Si hay al menos una fila que cumple la condición
        if not filtered_df.empty:
            # Crear el nuevo nombre del archivo
            new_filename = f'{base_name}_STRING_PP_SL.xlsx'
            output_path = os.path.join(output_directory, new_filename)
            
            # Guardar el DataFrame filtrado en un nuevo archivo Excel
            filtered_df.to_excel(output_path, index=False)
            print(f'Archivo guardado: {output_path}')
        else:
            # Imprimir mensaje si no hay filas que cumplan la condición
            print(f'El archivo {filename} no cumple la condición y no se ha creado un nuevo archivo.')

print("Proceso completado.")

El archivo ACOT9_2_interacciones_red-AC.xlsx no cumple la condición y no se ha creado un nuevo archivo.
Archivo guardado: Interacciones_interes\PSMB2_STRING_PP_SL.xlsx
Proceso completado.


## Selección de los identificadores de STRING de los posibles pares SL

In [82]:
import os
import shutil

# Define los directorios
interacciones_interes_dir = 'Interacciones_interes'
identificadores_string_dir = 'Identificadores_STRING'
identificadores_string_interes_dir = 'Identificadores_STRING_interes'

# Crea una lista con los nombres de los archivos en la carpeta Interacciones_interes quitando la terminación "_STRING_PP_SL.xlsxxx"
intereses_nombres = []
for filename in os.listdir(interacciones_interes_dir):
    if filename.endswith('_STRING_PP_SL.xlsx'):
        intereses_nombres.append(filename.replace('_STRING_PP_SL.xlsx', ''))

# Asegúrate de que el directorio Identificadores_STRING_interes exista
if not os.path.exists(identificadores_string_interes_dir):
    os.makedirs(identificadores_string_interes_dir)

# Mueve los archivos cuyo nombre corresponda con alguno de los de la lista
for filename in os.listdir(identificadores_string_dir):
    base_name = filename.replace('_2_identificadores_STRING.txt', '').replace('_identificadores_STRING.txt', '')
    if base_name in intereses_nombres:
        src = os.path.join(identificadores_string_dir, filename)
        dst = os.path.join(identificadores_string_interes_dir, filename)
        shutil.move(src, dst)
        print(f'Movido: {filename}')
        
print("Proceso completado.")

Movido: PSMB2_2_identificadores_STRING.txt
Proceso completado.


## Obtención de la imagen de red STRING
Con esta API puede recuperar una imagen de una red STRING de un vecindario que rodea una o más proteínas o pedir a STRING que muestre sólo la red de interacciones entre sus proteínas de entrada. Tanto los sabores de red (confianza y evidencia) como los tipos de red (funcional y física) son accesibles a través de la API. La API puede mostrar la imagen como un PNG (baja y alta resolución con canal alfa) o como un SVG (gráficos vectoriales que pueden ser modificados a través de scripts o en un software apropiado).

Para más información pulse [aquí](https://string-db.org/cgi/help.pl?subpage=api%23getting-string-network-image).

### Formatos de salida disponibles:

| Formato | Descripción |
|:--------|:------------|
| `image`| imagen PNG de red con canal alfa |
| `highres_image` | Imagen PNG de red de alta resolución con canal alfa |
| `svg` | Formato de gráficos vectoriales (SVG) |

### Parámetros disponibles:

#### Datos
| Parámetro | Descripción |
|:------|:------------|
| `identifiers`| Parámetro obligatorio para varios elementos, por ejemplo, DRD1_HUMAN%0dDRD2_HUMAN |
| `species` | Identificadores de taxones del NCBI (por ejemplo, Humano es 9606, véase: [organismos de STRING](https://string-db.org/cgi/input.pl?input_page_active_form=organisms)) |
| `add_color_nodes` | Añade nodos de color basados en puntuaciones a las proteínas de entrada |
| `add_white_nodes` | Añade nodos blancos basados en puntuaciones a las proteínas de entrada (añadidos después de los nodos de color) |
| `required_score`| Umbral de significación para incluir una interacción, un número entre 0 y 1000 (por defecto depende de la red) |
| `network_type`| Tipo de red: "functional" (funcional, por defecto), "physical" (física) |
| `caller_identity`| Su identificador para la API de STRING |

#### Visuales
| Parámetro | Descripción |
|:------|:------------|
| `network_flavor`| El estilo de las aristas en la red: "evidence" (evidencia, por defecto), "confidence" (confianza), acciones |
| `hide_node_labels` | Oculta todos los nombres de proteínas de la imagen (0 o 1) (por defecto: 0) |
| `hide_disconnected_nodes` | Oculta todas las proteínas que no están conectadas a ninguna otra proteína de su red (0 o 1) (por defecto: 0) |
| `show_query_node_labels` | Cuando se proporciona, utiliza los nombres presentados como etiquetas de proteínas en la imagen de red (0 o 1) (por defecto: 0) |
| `block_structure_pics_in_bubbles` | Desactiva las imágenes de estructura dentro de la burbuja (0 o 1) (por defecto: 0) |
| `flat_node_design` | Desactivar el diseño de burbujas 3D (0 o 1) (por defecto: 0) |
| `center_node_labels` | Centrar los nombres de las proteínas en los nodos (0 o 1) (por defecto: 0) |
| `custom_label_font_size` | Cambiar el tamaño de letra de los nombres de las proteínas (de 5 a 50) (por defecto: 12) |

Si consulta la API con una proteína, el parámetro «add_white_nodes» se establece automáticamente en 10, para que pueda ver el vecindario de interacciones de su proteína de consulta. Sin embargo, al igual que en la página web de STRING, cuando se consulta la API con más de una proteína, sólo se muestran las interacciones entre las proteínas introducidas. Por supuesto, siempre puede ampliar el vecindario de interacciones ajustando el parámetro «add_color/white_nodes» al valor deseado.

### Ejemplo de código python3

In [ ]:
#!/usr/bin/env python3

################################################################
## Para cada proteína de una lista guardar la imagen PNG de
## red STRING de sus 15 socios de interacción más seguros.
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la línea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
################################################################

import requests ## python -m pip install requests
from time import sleep

string_api_url = "https://version-11-5.string-db.org/api"
output_format = "image"
method = "network"

my_genes = ["YMR055C", "YFR028C",
            "YNL161W", "YOR373W",
            "YFL009W", "YBR202W"]


##
## Construir URL
##


request_url = "/".join([string_api_url, output_format, method])

## Para cada gen, llame a STRING

for gene in my_genes:

    ##
    ## Establecer parámetros
    ##

    params = {

        "identifiers" : gene, # su/s proteína/s
        "species" : 4932, # identificador NCBI de la especie
        "add_white_nodes": 15, # añadir 15 nodos blancos a mi proteína 
        "network_flavor": "confidence", # mostrar enlaces de confianza
        "caller_identity" : "www.awesome_app.org" # nombre de su aplicación

    }


    ##
    ## Llamar a STRING
    ##

    response = requests.post(request_url, data=params)

    ##
    ## Guardar la red en un archivo
    ##

    file_name = "%s_network.png" % gene
    print("Saving interaction network to %s" % file_name)

    with open(file_name, 'wb') as fh:
        fh.write(response.content)

    sleep(1)

### Código a ejecutar

In [9]:
#!/usr/bin/env python3

################################################################
## Para cada proteína de una lista guardar la imagen PNG de
## red STRING de sus 15 socios de interacción más seguros.
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la línea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
################################################################
#!pip3 install requests
import requests  # python -m pip install requests
import os
import glob
from time import sleep

# Definir el directorio de entrada y el directorio de salida
input_directory = 'Identificadores_STRING_interes'
output_directory = "Imagenes_red"
os.makedirs(output_directory, exist_ok=True)

# Leer genes desde el archivo
def read_genes_from_file(file_path):
    with open(file_path, 'r') as file:
        genes = [line.strip() for line in file if line.strip()]
    return genes

#string_api_url = "https://version-11-5.string-db.org/api"
output_format = "image"
method = "network"

def get_string_network(genes, output_file):
    request_url = "/".join([string_api_url, output_format, method])
    
    params = {
        "identifiers": genes,  # su/s proteína/s
        "species": 9606,  # identificador NCBI de la especie (p.ej., el del humano es 9606)
        "add_white_nodes": 5,  # añadir 5 nodos blancos a mi proteína
        "required_score": 700,  # umbral de significación para incluir una interacción, un número entre 0 y 1000 (por defecto depende de la red)
        "network_type": "functional",  # tipo de red: "functional" (por defecto), "physical"
        "network_flavor": "confidence",  # mostrar enlaces de confianza
        "hide_disconnected_nodes": "1",  # Oculta todas las proteínas que no están conectadas a ninguna otra proteína de su red (0 o 1) (por defecto:0).
        "caller_identity": "www.facebook.com/MSPCiencias/"  # nombre de su aplicación
    }
    
    try:
        response = requests.post(request_url, data=params)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for genes {genes}: {e}")
        return
    
    print(f"Saving interaction network to {output_file}")
    
    with open(output_file, 'wb') as fh:
        fh.write(response.content)

# Procesar todos los archivos .txt en el directorio de entrada
for input_file in glob.glob(os.path.join(input_directory, '*.txt')):
    my_genes = read_genes_from_file(input_file)
    joined_genes = "%0d".join(my_genes)
    
    # Generar el nombre del archivo de salida
    base_name = os.path.basename(input_file)
    output_file_name = base_name.replace('_identificadores_STRING', '').replace('.txt', '') + '_red.png'
    output_file_path = os.path.join(output_directory, output_file_name)
    
    # Hacer una petición conjunta para todos los genes
    get_string_network(joined_genes, output_file_path)
    
    # Esperar un segundo antes de la siguiente petición
    sleep(1)
    
print("Proceso completado.")

Saving interaction network to Imagenes_red\TP53_2_red.png
Proceso completado.


## Recuperación de anotaciones funcionales.
STRING mapea varias bases de datos en sus proteínas, esto incluye: [Gene Ontology](https://www.geneontology.org/), [rutas de KEGG](http://www.genome.jp/kegg/pathway.html), [palabras clave de UniProt](https://www.uniprot.org/help/keywords), publicaciones de [PubMed](https://www.ncbi.nlm.nih.gov/pubmed/), [dominios de Pfam](http://pfam.xfam.org/), [dominios de InterPro](https://www.ebi.ac.uk/interpro/), y [dominios de SMART](http://smart.embl.de/). Puede recuperar todas estas anotaciones (y no sólo el subconjunto enriquecido) para sus proteínas a través de esta API. Debido al gran tamaño potencial de las asignaciones PubMed (Publicaciones de Referencia), no se enviarán por defecto, pero puede volver a activarlas especificando el parámetro 'allow_pubmed=1'.

Nota: las anotaciones KEGG no están disponibles debido a las restricciones de la licencia KEGG.

Para información adicional pulse [aquí](https://string-db.org/cgi/help.pl?subpage=api%23retrieving-functional-annotation).

### Formatos de salida disponibles:

| Formato | Descripción |
|:--------|:------------|
| `tsv`| Valores separados por tabuladores, con una línea de encabezamiento |
| `tsv-no-header` | Valores separados por tabuladores, sin línea de encabezamiento |
| `json` | Formato JSON |
| `xml` | Formato XML |

### Parámetros disponibles:

| Parámetro | Descripción |
|:------|:------------|
| `identifiers`| Parámetro obligatorio para varios elementos, por ejemplo, DRD1_HUMAN%0dDRD2_HUMAN |
| `species` | Identificadores de taxones del NCBI (por ejemplo, Humano es 9606, véase: [organismos de STRING](https://string-db.org/cgi/input.pl?input_page_active_form=organisms)) |
| `allow_pubmed` | "1" para imprimir también las anotaciones de PubMed además de otras categorías, por defecto es "0" |
| `only_pubmed` | "1" para imprimir sólo las anotaciones de PubMed, por defecto es "0" |
| `caller_identity`| Su identificador para la API de STRING |

### Ejemplo de llamada (anotación funcional de la CDK1 humana):
https://string-db.org/api/tsv/functional_annotation?identifiers=cdk1

### Código a ejecutar

In [84]:
#!/usr/bin/env python3

##################################################################
## Para la lista dada de proteinas imprime sus correspondientes anotaciones
## (y no sólo el subconjunto enriquecido)
##
## Requiere el módulo requests:
## escriba "python -m pip install requests" en la linea de comandos (win)
## o terminal (mac/linux) para instalar el módulo
##################################################################
import os
#!pip3 install requests
import requests
#!pip3 install openpyxl
import openpyxl
import time

# Carpeta de entrada y salida
input_folder = "Identificadores_STRING_interes"
output_folder = "Anotaciones_funcionales"

# Verificar si la carpeta de salida existe, si no, crearla
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Obtener la lista de archivos .txt en la carpeta de entrada
txt_files = [f for f in os.listdir(input_folder) if f.endswith('.txt')]

# Iterar sobre cada archivo .txt
for txt_file in txt_files:
    try:
        # Leer los identificadores del archivo .txt
        file_path = os.path.join(input_folder, txt_file)
        with open(file_path, "r") as file:
            my_genes = file.read().strip().split("\t")

        # Definir la URL de la API de STRING
        #string_api_url = "https://version-12-0.string-db.org/api"
        output_format = "tsv-no-header"
        method = "functional_annotation"
        request_url = "/".join([string_api_url, output_format, method])

        # Establecer parámetros
        params = {
            "identifiers": "%0d".join(my_genes), # su/s proteína/s
            "species": 9606, # identificador NCBI de la especie (p.ej., el del humano es 9606) 
            "caller_identity": "www.facebook.com/MSPCiencias/" # nombre de su aplicación
        }

        # Llamar a STRING
        response = requests.post(request_url, data=params)

        # Verificar si la respuesta fue exitosa
        response.raise_for_status()

        # Crear un nuevo archivo Excel
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["Categoría", "Término", "Nº de genes", "Proporción en el conjunto", "Identificador de taxón del NCBI",
        "Genes de entrada", "Nombres preferidos", "Descripción"])

        # Procesar la respuesta y escribir en el archivo Excel
        for line in response.text.strip().split("\n"):
            l = line.strip().split("\t")
            if len(l) >= 8:  # Verificar si hay suficientes columnas en la línea
                category, term = l[0], l[1]
                number_of_genes = int(l[2])
                ratio_in_set = float(l[3])
                ncbiTaxonId, inputGenes, preferredNames, description = l[4], l[5], l[6], l[7]
                ws.append([category, term,  number_of_genes, "%.2f" % ratio_in_set, ncbiTaxonId, inputGenes, preferredNames, description])

        # Guardar el archivo Excel
        excel_file_name = txt_file.replace("_identificadores_STRING.txt", "_anotaciones_funcionales.xlsx")
        excel_file_path = os.path.join(output_folder, excel_file_name)
        wb.save(excel_file_path)
        print("Los resultados del archivo {} se han guardado en el archivo Excel: {}".format(txt_file, excel_file_path))
    except Exception as e:
        print("Se produjo un error al procesar el archivo {}: {}".format(txt_file, str(e)))
        continue

    # Hacer una pausa de 1 segundo antes de procesar el siguiente archivo
    time.sleep(1)

print("Proceso completado.")

Los resultados del archivo PSMB2_2_identificadores_STRING.txt se han guardado en el archivo Excel: Anotaciones_funcionales\PSMB2_2_anotaciones_funcionales.xlsx
Proceso completado.


### Campos de salida:

| Campo | Descripción |
|:------|:------------|
| `category`| Categoría del término (p. ej., proceso GO, vías KEGG) |
| `term` | Término enriquecido (término GO, dominio o ruta) |
| `number_of_genes` | Número de genes de su lista de entrada con el término asignado |
| `ratio_in_set` | Proporción de las proteínas de su lista de entrada con el término asignado |
| `ncbiTaxonId`| Identificador de taxón del NCBI |
| `inputGenes`| Nombres de los genes a partir de su entrada |
| `preferredNames`| Nombres comunes de las proteínas (en el mismo orden que los genes introducidos) |
| `description`| Descripción del término enriquecido |

## Vaciado de las carpetas de archivos
Tras copiar los archivos que considere de interés a otra ubicación, puede ejecutar este código para borrar todos los archivos presentes en las carpetas indicadas en la "Lista de las carpetas".

In [85]:
import os

# Lista de las carpetas
carpetas = [
    "Firmas_genes",
    "Identificadores_STRING",
    "Identificadores_STRING_interes",
    "Imagenes_red",
    "Interacciones_alta_confianza",
    "Interacciones_interes",
    "Interacciones_red",
    "Resultados_EIPP",
    "Anotaciones_funcionales"
]

def eliminar_archivos(carpeta):
    # Listar todos los elementos en la carpeta
    for filename in os.listdir(carpeta):
        file_path = os.path.join(carpeta, filename)
        try:
            # Verificar si es un archivo y eliminarlo
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f'Archivo {file_path} eliminado')
            # Si es un directorio, lo ignoramos
        except Exception as e:
            print(f'No se pudo eliminar {file_path}. Razón: {e}')

# Eliminar archivos en cada carpeta
for carpeta in carpetas:
    eliminar_archivos(carpeta)
    
print("Proceso completado.")

Archivo Identificadores_STRING\ACOT9_2_identificadores_STRING.txt eliminado
Archivo Identificadores_STRING\MMP1_identificadores_STRING.txt eliminado
Archivo Identificadores_STRING\MMP3_2_identificadores_STRING.txt eliminado
Archivo Identificadores_STRING\NETO2_identificadores_STRING.txt eliminado
Archivo Identificadores_STRING\SLC16A1_2_identificadores_STRING.txt eliminado
Archivo Identificadores_STRING_interes\PSMB2_2_identificadores_STRING.txt eliminado
Archivo Imagenes_red\PSMB2_2_red.png eliminado
Archivo Interacciones_alta_confianza\ACOT9_2_interacciones_red-AC.xlsx eliminado
Archivo Interacciones_alta_confianza\PSMB2_2_interacciones_red-AC.xlsx eliminado
Archivo Interacciones_interes\PSMB2_STRING_PP_SL.xlsx eliminado
Archivo Anotaciones_funcionales\PSMB2_2_anotaciones_funcionales.xlsx eliminado
Proceso completado.
